Create Inference Compute Environment

In [ ]:
%%writefile components/conda.yml
name: text-to-image-pipeline-env
channels:
  - conda-forge
dependencies:
  - python=3.10
  - pip
  - pip:
    - azureml-evaluate-mlflow==0.0.35
    - inference-schema[numpy-support]==1.3.0
    - pyarrow
    - torch


In [ ]:
%%writefile components/text-to-image-pipeline-env.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: text-to-image-pipeline-cuda-env
image: mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.8-cudnn8-ubuntu22.04
conda_file: conda.yml
description: Environment created from a Docker image plus Conda environment.

In [ ]:
!az ml environment create --file components/text-to-image-pipeline-env.yml

Create Pipeline Component that will evaluate text using an LLM model

In [ ]:
%%writefile components/model_scorer.py

# Read all the csvs in the data folder into a pandas dataframe
import glob
import os
import pandas as pd
import argparse
import mlflow
from pathlib import Path
import torch

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--mlflow_model", type=str, help="mlflow model")
    parser.add_argument("--prompts", type=str, help="path to prompts input")
    parser.add_argument("--predictions", type=str, help="path to predictions output")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(f"Got prompts: {args.prompts}")

    batch_df = pd.read_json(args.prompts, lines=True)

    model = mlflow.pyfunc.load_model(args.mlflow_model)
    
    print(f"Loaded model object: {model}")

    batch_df['input_string'] = batch_df['text'].astype(str)

    batch_df = batch_df[["input_string"]]
    
    predictions = model.predict(batch_df)

    print(f"Predictions DF: {predictions.columns}")

    print(f"Predictions: {predictions.head()}")

    for i in range(len(predictions[0])):
        print("********************************************************************")
        p_clean = predictions[0][i].replace('\n', '')
        print(f"Input String{i}: {batch_df['input_string'][i]}")
        print(f"Prediction {i}: {p_clean}")
        
        ###############################################
        # YOUR custom handling of the prediction here
        ###############################################

        print("********************************************************************")

    predictions_df = pd.DataFrame(predictions[0])
    predictions_df["input_string"] = batch_df["input_string"]

    predictions_df.to_csv(args.predictions, index=False, header=False)
    
    mlflow.end_run()

if __name__ == "__main__":
    main()

In [ ]:
%%writefile components/model_scorer.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
name: your_custom_model_scorer
code: .
command: >-
  python model_scorer.py 
  --mlflow_model ${{inputs.mlflow_model}}
  --prompts ${{inputs.prompts}}
  --predictions ${{outputs.predictions}}
inputs:
  mlflow_model: 
    type: uri_folder
  prompts: 
    type: uri_file
outputs:
  predictions:
    type: uri_file
environment: azureml:custom-env-1@latest
compute: azureml:FT-Standard-NC24s-v3
display_name: your_custom_model_scorer
experiment_name: text-to-image-pipeline
description: Score an MLFlow model

In [ ]:
!az ml component create -f components/model_scorer.yml